In [1]:
import pandas as pd
#import joblib
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
#from costcla.models import CostSensitiveDecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import SMOTE
import math
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

C:\Users\DELL\Anaconda3\lib\site-packages\dask\dataframe\utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train = pd.read_csv('D:\\Data Storm 2.0\\Hotel-A-train.csv')
validation = pd.read_csv('D:\\Data Storm 2.0\\Hotel-A-validation.csv')
test = pd.read_csv('D:\\Data Storm 2.0\\Hotel-A-test.csv')

In [104]:
train.columns

Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type', 'Expected_checkin',
       'Expected_checkout', 'Booking_date',
       'time between booking date and check in date', 'time of stay', 'Adults',
       'Children', 'Babies', 'Meal_Type', 'Visted_Previously',
       'Previous_Cancellations', 'Deposit_type', 'Booking_channel',
       'Required_Car_Parking', 'Reservation_Status', 'Use_Promotion',
       'Discount_Rate', 'Room_Rate', 'Final_Room_Rate', 'Rooms_booked',
       'Final_rate'],
      dtype='object')

In [3]:
#Creating new features --> Number of rooms booked
people = train['Adults']+train['Children']
rooms= people/5
train['Rooms_booked'] = np.ceil(rooms)

In [4]:
#Creating new features --> Number of rooms booked
people2 = validation['Adults']+validation['Children']
rooms2 = people2/5
validation['Rooms_booked'] = np.ceil(rooms2)

#Creating new features --> Number of rooms booked
people3 = test['Adults']+test['Children']
rooms3 = people3/5
test['Rooms_booked'] = np.ceil(rooms3)

In [5]:
#Creating feature final room rate
train['Final_Room_Rate'] = train['Room_Rate'] - (train['Room_Rate']*(train['Discount_Rate']/100))
validation['Final_Room_Rate'] = validation['Room_Rate'] - (validation['Room_Rate']*(validation['Discount_Rate']/100))
test['Final_Room_Rate'] = test['Room_Rate'] - (test['Room_Rate']*(test['Discount_Rate']/100))

In [8]:
#Creating feature Final Rate
train['Final_rate'] = train['Final_Room_Rate']*train['Rooms_booked']*train['time of stay']
validation['Final_rate'] = validation['Final_Room_Rate']*validation['Rooms_booked']*validation['time of stay']
test['Final_rate'] = test['Final_Room_Rate']*test['Rooms_booked']*test['time of stay']

In [110]:
test.head()

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Use_Promotion,Discount_Rate,Room_Rate,Rooms_booked,Final_Room_Rate,Final_rate
0,62931593,0,52,2,1,0,2,1,11/18/2016,11/19/2016,...,0,0,1,1,1,10,153,2.0,137.70,275.40
1,70586099,0,47,2,1,0,0,0,11/18/2016,11/19/2016,...,0,0,2,0,0,0,210,1.0,210.00,210.00
2,4230648,0,28,1,1,2,0,1,4/28/2017,5/1/2017,...,0,0,0,0,1,5,117,1.0,111.15,333.45
3,25192322,0,65,3,2,0,2,0,11/18/2016,11/20/2016,...,0,0,2,1,1,10,107,1.0,96.30,192.60
4,80931528,1,45,0,0,0,2,1,11/18/2016,11/20/2016,...,0,2,0,0,0,0,119,1.0,119.00,238.00


In [131]:
train.tail()

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Room_Rate,Final_Room_Rate,Rooms_booked,Final_rate
27494,15645505,0,49,0,3,2,3,1,8/31/2016,9/1/2016,...,0,2,1,1,1,10,100,90.0,1.0,90.0
27495,56414525,0,49,1,0,0,2,0,8/31/2016,9/1/2016,...,2,0,1,2,1,40,194,116.4,1.0,116.4
27496,52023375,0,49,2,2,1,2,0,9/1/2016,9/2/2016,...,0,1,1,1,1,5,202,191.9,1.0,191.9
27497,98041387,1,65,0,0,0,2,0,8/31/2016,9/1/2016,...,0,2,0,0,1,20,157,125.6,1.0,125.6
27498,55243988,0,48,3,0,0,1,2,8/31/2016,9/1/2016,...,0,2,0,2,0,0,210,210.0,1.0,210.0


In [10]:
#Obtaining a df of categorical variables to encode
obj_df = train.select_dtypes(include=['object']).copy()
obj_df = obj_df.drop(['Expected_checkin',
       'Expected_checkout', 'Booking_date'],axis=1)
obj_df.tail()

,Gender,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion
27494,F,African American,Mid-School,<25K,West,City Hotel,HB,Yes,Yes,No Deposit,Online,Yes,Check-In,Yes
27495,F,Asian American,College,25K --50K,South,Airport Hotels,BB,No,No,Refundable,Agent,Yes,No-Show,Yes
27496,F,Latino,High-School,50K -- 100K,South,Airport Hotels,HB,No,No,No Deposit,Direct,Yes,Check-In,Yes
27497,M,African American,College,25K --50K,South,Airport Hotels,FB,No,No,No Deposit,Online,No,Canceled,Yes
27498,F,caucasian,College,25K --50K,North,Resort,HB,No,No,No Deposit,Online,No,No-Show,No


In [11]:
#Label encoding the train set
label = LabelEncoder()
for col in obj_df.columns:
    train[col] = label.fit_transform(train[col])
    #return train

In [12]:
#label encoding validation set
for col in obj_df.columns:
    validation[col] = label.fit_transform(validation[col])

In [14]:
#Label encoding test set
for col in obj_df.columns.drop('Reservation_Status'):
    test[col] = label.fit_transform(test[col])
    #return train

In [16]:
train.dtypes

Reservation-id                                   int64
Gender                                           int32
Age                                              int64
Ethnicity                                        int32
Educational_Level                                int32
Income                                           int32
Country_region                                   int32
Hotel_Type                                       int32
Expected_checkin                                object
Expected_checkout                               object
Booking_date                                    object
time between booking date and check in date      int64
time of stay                                     int64
Adults                                           int64
Children                                         int64
Babies                                           int64
Meal_Type                                        int32
Visted_Previously                                int32
Previous_C

In [17]:
#Scaling the dataset using Standard Scaler
standard = StandardScaler()
#to_scale = train.select_dtypes(include=['int64','float64'])

In [31]:
#Scaling the data in the train, validation and test sets
train[['Age', 'time between booking date and check in date', 'time of stay','Adults', 'Children', 'Babies', 'Discount_Rate', 'Room_Rate',
       'Final_Room_Rate', 'Rooms_booked', 'Final_rate']] = standard.fit_transform(train[['Age', 'time between booking date and check in date', 'time of stay',
       'Adults', 'Children', 'Babies', 'Discount_Rate', 'Room_Rate',
       'Final_Room_Rate', 'Rooms_booked', 'Final_rate']])

validation[['Age', 'time between booking date and check in date', 'time of stay','Adults', 'Children', 'Babies', 'Discount_Rate', 'Room_Rate',
       'Final_Room_Rate', 'Rooms_booked', 'Final_rate']] = standard.fit_transform(validation[['Age', 'time between booking date and check in date', 'time of stay',
       'Adults', 'Children', 'Babies', 'Discount_Rate', 'Room_Rate',
       'Final_Room_Rate', 'Rooms_booked', 'Final_rate']])

In [32]:
#Splitting train set to x and y values
x = train.drop(['Reservation_Status','Reservation-id','Expected_checkin',
       'Expected_checkout', 'Booking_date','Final_Room_Rate','Rooms_booked'],axis=1)
y = train['Reservation_Status']

In [33]:
#Splitting validation set to x and y values
x_val = validation.drop(['Reservation_Status','Reservation-id','Expected_checkin',
       'Expected_checkout', 'Booking_date','Final_Room_Rate','Rooms_booked'],axis=1)
y_val = validation['Reservation_Status']

In [34]:
#dropping columns from test set
x_test = test.drop(['Reservation-id','Expected_checkin',
       'Expected_checkout', 'Booking_date','Final_Room_Rate','Rooms_booked'],axis=1)

In [35]:
#Applying SMOTE sampling to the train set to balance the dataset
smote = SMOTE()
x_train, y_train = smote.fit_resample(x,y)

In [123]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)
y_pred_tree = clf.predict(x_val)

In [124]:
dt_f1_score = f1_score(y_val, y_pred_tree, average='weighted')
dt_f1_score

0.42927571617536314

In [36]:
### Fitting XGBoost Model
xgb = XGBClassifier(random_state=2)
xgb.fit(x_train,y_train)

## Making predctions
xgb_predict = xgb.predict(x_val)


In [37]:
## F1 Score
xgb_fscore = f1_score(y_val,xgb_predict,average='weighted')
print(xgb_fscore)

0.1394483387496824


In [132]:
#### Parameter tuning
params_grid = {
    'max_depth':[1,2,3,4,5,6],
    'n_estimators':[5,10,25,50,100],
    'learning_rate': np.linspace(0.01,0.5,num=8,endpoint=False)
}

In [133]:
params_fixd = {
    'objective': 'multi:softmax',
}

In [134]:
best_grid = GridSearchCV(
    estimator=XGBClassifier(**params_fixd,random_state=2),
#    estimator=GradientBoostingClassifier(random_state=2),
#    estimator=AdaBoostClassifier(random_state=2),
    param_grid = params_grid,
    scoring='f1_weighted'
)

In [ ]:
## Parameter tuning for extreme gradient boosting XGBoost
best_grid.fit(x_train,y_train)
print(best_grid.best_score_,best_grid.best_params_)

In [125]:
y_test3 = xgb.predict(x_test)

In [126]:
y_test3

array([1, 2, 2, ..., 0, 1, 0], dtype=int64)

In [127]:
test_df = pd.DataFrame(y_test3, columns = ['Reservation_Status'])
test_df['Reservation-id'] = test['Reservation-id']
column_titles = ['Reservation-id','Reservation_Status']
test_df = test_df.reindex(columns=column_titles)

In [129]:
test_df.tail()

,Reservation-id,Reservation_Status
4313,39110574,1
4314,5496554,1
4315,59004046,0
4316,65838682,1
4317,14474101,0


In [130]:
test_df.to_csv('submission_3.csv')